In [1]:
"""
Relevance Detection
"""

'\nRelevance Detection\n'

In [2]:
import pandas as pd
import nltk
import numpy as np
import preprocessing
import importlib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
from sklearn.metrics import confusion_matrix
import score
from sklearn.preprocessing import StandardScaler

In [426]:
#run this cell to reload the preprocessing module
importlib.reload(preprocessing)

<module 'preprocessing' from '/Users/dannyyang/Documents/GitHub/Insights-FakeNews/preprocessing.py'>

In [219]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [5]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [372]:
stances_tr, stances_val = preprocessing.train_test_split(train_bodies, train_stances)
stances_tr.shape, stances_val.shape

((39375, 3), (10597, 3))

In [ ]:
# this one takes a while!
idf = preprocessing.build_idf(train_bodies, stances_tr)

In [409]:
#this is just a comparison between using IDF score and not using IDF score - not related to the model
#change the body id to see
body = preprocessing.get_body(1369, train_bodies)
#no IDF
processed2 = preprocessing.process_body(body)
print(processed2['common_nouns'],processed2['common_verbs'])

#with IDF
processed = preprocessing.process_body(body, idf)
print(processed['common_nouns'],processed['common_verbs'])

['attack', 'kobani', 'victim', 'weapon', 'border', 'report', 'doctor', 'burn', 'eye', 'isi'] ['said', 'used', 'carried', 'battling', 'reporting', 'swollen', 'told', 'spoke', 'confirmed', 'ahmed']
['kobani', 'attack', 'moussa', 'burn', 'headache', 'victim', 'suffocation', 'doctor', 'eye', 'throat'] ['swollen', 'battling', 'used', 'said', 'spoke', 'carried', 'ahmed', 'reporting', 'led', 'surrendered']


In [414]:
#this takes a while!
body_info = preprocessing.process_bodies(train_bodies, idf)

processed 100
processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
done! processed 1683


In [419]:
feats_list = [
    'shared_nouns',
    'shared_verbs',
    'shared_bigrams',
    'shared_trigrams',
    'shared_tokens',
#     'sentiment_pos',
#     'sentiment_neg',
#     'sentiment_neu',
#     'sentiment_compound',

    'shared_nouns_fst',
    'shared_verbs_fst',
    'shared_bigrams_fst',
    'shared_trigrams_fst',
    'shared_tokens_fst',
#     'sentiment_pos_fst',
#     'sentiment_neg_fst',
#     'sentiment_neu_fst',
#     'sentiment_compound_fst',

    'shared_nouns_sig',
    'shared_verbs_sig',
    'shared_bigrams_sig',
    'shared_trigrams_sig',
    'shared_tokens_sig',
#     'sentiment_pos_sig',
#     'sentiment_neg_sig',
#     'sentiment_neu_sig',
#     'sentiment_compound_sig',
]

In [427]:
# this one takes a while also!

#train data
data_feats = [preprocessing.get_feats(i, body_info) for i in stances_tr.values]
train_df = pd.DataFrame()
for i in feats_list:
    train_df[i] = [x[i] for x in data_feats]

#val data
val_feats = [preprocessing.get_feats(i, body_info) for i in stances_val.values]
val_df = pd.DataFrame()
for i in feats_list:
    val_df[i] = [x[i] for x in val_feats]

In [428]:
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_trigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_trigrams_fst,shared_tokens_fst,shared_nouns_sig,shared_verbs_sig,shared_bigrams_sig,shared_trigrams_sig,shared_tokens_sig
0,2,0,0,0,6,1,0,0,0,3,1,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,3,0,0,0,0,0,1,0,0,0,1
3,5,0,0,0,12,3,0,2,1,8,5,0,7,6,9
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [429]:
train_df['label'] = [-1 if x == "unrelated" else 1 for x in list(stances_tr['Stance'])]
val_df['label'] = [-1 if x == "unrelated" else 1 for x in list(stances_val['Stance'])]
Counter(train_df['label'])

Counter({-1: 28772, 1: 10603})

In [430]:
scaler = StandardScaler()

for i in feats_list:
    train_df[i] = scaler.fit_transform(train_df[i].values.reshape(-1,1))
    val_df[i] = scaler.fit_transform(val_df[i].values.reshape(-1,1))

/Users/dannyyang/Library/Python/3.6/lib/python/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [431]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
model = LogisticRegression()
model.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [432]:
tr_acc = model.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

95.49% training accuracy
96.05% validation accuracy


In [433]:
#get coefficients
[(feats_list[i],model.coef_[0][i]) for i in list(range(len(feats_list)))]

[('shared_nouns', 1.0436195894034368),
 ('shared_verbs', 0.17090579112350138),
 ('shared_bigrams', 0.0),
 ('shared_trigrams', 0.0),
 ('shared_tokens', 2.5444769305638357),
 ('shared_nouns_fst', -0.59776984389062682),
 ('shared_verbs_fst', -0.088235385011290524),
 ('shared_bigrams_fst', -0.60259625801276784),
 ('shared_trigrams_fst', -0.035107224032494944),
 ('shared_tokens_fst', 2.3794049411145468),
 ('shared_nouns_sig', -0.17922469503414037),
 ('shared_verbs_sig', -0.15152716694507024),
 ('shared_bigrams_sig', -0.49102116510446042),
 ('shared_trigrams_sig', 0.61343327702741157),
 ('shared_tokens_sig', 0.89173118714318367)]

In [434]:
true_label = val_df.iloc[:,-1]
prediction = model.predict(val_df.iloc[:,:-1])
matrix = confusion_matrix(true_label,prediction)
print('confusion matrix: \n{}\n'.format(matrix))
tn1, fp1, fn1, tp1 = matrix.ravel()

confusion matrix: 
[[7637  136]
 [ 283 2541]]



In [436]:
#use FNC scorer to generate score report
label_prediction = ["discuss" if i == 1 else "unrelated" for i in prediction]
label_actual = pd.DataFrame(stances_val)['Stance']
score.report_score(label_actual, label_prediction)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |     0     |    626    |    71     |
-------------------------------------------------------------
| disagree  |     0     |     0     |    168    |    24     |
-------------------------------------------------------------
|  discuss  |     0     |     0     |   1747    |    188    |
-------------------------------------------------------------
| unrelated |     0     |     0     |    136    |   7637    |
-------------------------------------------------------------
Score: 3854.75 out of 4767.25	(80.85898578845246%)


80.85898578845246